In [1]:
# Librerias
%matplotlib notebook
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
print(torch.cuda.is_available())
from torch.utils.data import DataLoader, Subset
import sklearn.model_selection
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import pandas as pd
import torchaudio
from sklearn import metrics as mt
from ignite.engine import  Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, Metric, ClassificationReport
from ignite.handlers import ModelCheckpoint, EarlyStopping
from codes import utils, metric
from codes.models import SEDnet
from codes.dataset2 import WetSoundDataset
from sklearn.model_selection import KFold
import torch.nn as nn
from torch.nn.functional import sigmoid
from torch.utils.tensorboard import SummaryWriter

True


# Test

In [2]:
main = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
AUDIO_DIR = os.path.join(main,"features/test_set/")
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
NUM_SAMPLES = 1024  #Cantidad de secuencias , no largo de secuencia 12920/40
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40
LEN_MBE = round(LEN_SAMPLES/int(HOP*NUM_SAMPLES))
frames_1_sec = int(SAMPLE_RATE/(N_FFT/2.0))
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

Using device cuda


In [4]:
wet = WetSoundDataset(AUDIO_DIR,
                        LEN_MBE,
                        NUM_SAMPLES,
                        __class_labels,
                        device)
print(f"There are {len(wet)} samples in the dataset.")
test_loader = DataLoader(wet, shuffle=False, batch_size=1)

There are 3432 samples in the dataset.


## Cargar modelos entrenados

In [5]:
path_models = "models/"
results = dict()
for fold_model in os.listdir(path_models):
    filter_file = fold_model.split(".")
    if len(filter_file)>1:
        if filter_file[2]=="pt":
            print(fold_model)
            model = SEDnet(len(__class_labels))
            model.load_state_dict(torch.load(path_models+fold_model))
            model.to(device)

            #pred = list()
            #targets = list()
            i=0
            for data,label in test_loader:
                with torch.no_grad():
                    pred_i = model.forward(data)
                label = label.to('cpu')
                pred_i = pred_i.to('cpu')
                pred_i,label = metric.thresholded_output_transform([pred_i,label])
                pred_i = pred_i.view(pred_i.shape[1]*pred_i.shape[0],pred_i.shape[2])
                label = label.view(label.shape[1]*label.shape[0],label.shape[2])
                if i == 0:
                    pred = pred_i
                    targets = label
                else:
                    pred = torch.cat((pred,pred_i))
                    targets = torch.cat((targets,label))
                i+=1
                del pred_i
                del data
                del label
                torch.cuda.empty_cache()

            pred = pred.numpy()
            targets = targets.numpy()

            metrics_data = metric.compute_scores_orig(pred, targets,frames_1_sec)

            results[fold_model]=metrics_data

best_SEDnet1_val_error=-0.2207.pt


/home/vpoblete/miniconda3/envs/ia_257/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/home/vpoblete/miniconda3/envs/ia_257/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


best_SEDnet3_val_error=-0.2021.pt
best_SEDnet0_val_error=-0.2197.pt
best_SEDnet2_val_error=-0.2079.pt


In [6]:
results

{'best_SEDnet1_val_error=-0.2207.pt': {'f1_overall_1sec': 0.754,
  'er_overall_1sec': 0.485},
 'best_SEDnet3_val_error=-0.2021.pt': {'f1_overall_1sec': 0.754,
  'er_overall_1sec': 0.492},
 'best_SEDnet0_val_error=-0.2197.pt': {'f1_overall_1sec': 0.754,
  'er_overall_1sec': 0.502},
 'best_SEDnet2_val_error=-0.2079.pt': {'f1_overall_1sec': 0.751,
  'er_overall_1sec': 0.496}}